In [41]:
from backtest.repository.finance.finance_repo import FinanceRepo
%load_ext autoreload
%autoreload 2

from backtest.domains.strategy import StockData
from backtest.domains.strategy import Strategy
from backtest.domains.strategy_result import StrategyResult,StrategyResultColumnType
from backtest.domains.backtest import Backtest
from backtest.repository.webrepo.bithumb_repo import BithumbRepo
from backtest.repository.webrepo.upbit_repo import UpbitRepo
from backtest.repository.webrepo.binance_repo import BinanceRepo
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.use_cases.strategy_execute import rsi_function, rsi_big_stock_function
from backtest.use_cases.backtest_execute import backtest_execute

import pandas as pd

request = build_stock_data_from_repo_request(
    filters={'order__eq': 'ALGO', 'from__eq': '2019-01-01'})
response = stockdata_from_repo(BithumbRepo(), request=request)

request2 = build_stock_data_from_repo_request(
    filters={'order__eq': 'BTC', 'from__eq': '2019-01-01'})
response2 = stockdata_from_repo(BithumbRepo(), request=request2)

request3 = build_stock_data_from_repo_request(
    filters={'order__eq': 'ETH', 'from__eq': '2019-01-01'})
response3 = stockdata_from_repo(BithumbRepo(), request=request3)

request5 = build_stock_data_from_repo_request(
    filters={'order__eq': 'XRP', 'from__eq': '2019-01-01'})
response5 = stockdata_from_repo(BithumbRepo(), request=request5)

request4 = build_stock_data_from_repo_request(
    filters={'order__eq': 'US500', 'from__eq': '2019-01-01'})
response4 = stockdata_from_repo(FinanceRepo(), request=request4)

stockdata1 = response.value
stockdata2 = response2.value
stockdata3 = response3.value
stockdata4 = response4.value
stockdata5 = response5.value


#strategy1 = Strategy(name='rsi_function', function=rsi_function,weight=1,options={'rolling':100})
strategy1 = Strategy(name='snp_big_stock', function=rsi_big_stock_function,
                    weight=3, options={'big_stock': stockdata4, 'rolling': 100})
strategy2 = Strategy(name='sma_big_stock', function=rsi_big_stock_function,
                    weight=2, options={'big_stock': stockdata2, 'rolling': 90})
strategy3 = Strategy(name='sma_self', function=rsi_function,
                    weight=1, options={'rolling': 7})
backtest = Backtest(strategy_list=[strategy1,strategy2,strategy3], stockdata_list=[stockdata1,stockdata3])
backtest_result = backtest_execute(backtest).value
backtest_result.value.head()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,total_profit,stock_bucket,total_stock_count,stock_count
date,,,,
2019-01-01,0.0,[],0,{}
2019-01-02,0.0,[],0,{}
2019-01-03,0.0,[],0,{}
2019-01-04,0.0,[],0,{}
2019-01-05,0.0,[],0,{}


In [42]:
backtest_result.value['total_profit'].expanding().sum()

date
2019-01-01    0.000000
2019-01-02    0.000000
2019-01-03    0.000000
2019-01-04    0.000000
2019-01-05    0.000000
                ...   
2023-02-25    1.898085
2023-02-26    1.898085
2023-02-27    1.898085
2023-02-28    1.898085
2023-03-01    1.898085
Freq: D, Name: total_profit, Length: 1521, dtype: float64

In [132]:
pd.options.display.max_colwidth = 1000
backtest_result.value[['total_profit','total_stock_count','stock_count']].to_csv(
    'result.csv')


AttributeError: 'dict' object has no attribute 'data'

AttributeError: 'dict' object has no attribute 'data'

In [151]:
from backtest.repository.finance.finance_repo import FinanceRepo
request4 = build_stock_data_from_repo_request(
    filters={'order__eq': 'US500', 'from__eq': '2019-01-01'})
response4 = stockdata_from_repo(FinanceRepo(), request=request4)
stockdata4 = response4.value
stockdata4.data


,open,high,low,close,volume
Date,,,,,
2019-01-02,2476.959961,2519.489990,2467.469971,2510.030029,3733160000
2019-01-03,2491.919922,2493.139893,2443.959961,2447.889893,3858830000
2019-01-04,2474.330078,2538.070068,2474.330078,2531.939941,4234140000
2019-01-07,2535.610107,2566.159912,2524.560059,2549.689941,4133120000
2019-01-08,2568.110107,2579.820068,2547.560059,2574.409912,4120060000
...,...,...,...,...,...
2023-02-21,4052.350098,4052.350098,3995.189941,3997.340088,4121590000
2023-02-22,4001.830078,4017.370117,3976.899902,3991.050049,4079320000
2023-02-23,4018.600098,4028.300049,3969.189941,4012.320068,3952940000


In [ ]:
stockdate